In [1]:
import os
os.environ['POLARS_MAX_THREADS'] = '8'
import pandas as pd
import polars as pl
from pathlib import Path

from esgpt_task_querying import main
from EventStream.data.dataset_polars import Dataset

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', 100)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

data_path = '../MIMIC_ESD_new_schema_08-31-23-1'
os.getcwd()

/home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/justinxu/esgpt/ESGPTTaskQuerying/to_organize'

In [2]:
DATA_DIR = Path(data_path)
ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.filter(~pl.all_horizontal(pl.all().is_null()))
dynamic_measurements_df = ESD.dynamic_measurements_df.filter(
    ~pl.all_horizontal(pl.all().is_null())
)

ESD_data = (
    events_df.join(dynamic_measurements_df, on="event_id", how="left")
    .drop(["event_id"])
    .sort(by=["subject_id", "timestamp", "event_type"])
)

if ESD_data["timestamp"].dtype != pl.Datetime:
    ESD_data = ESD_data.with_columns(
        pl.col("timestamp")
        .str.strptime(pl.Datetime, format="%m/%d/%Y %H:%M")
        .cast(pl.Datetime)
    )

Updating config.save_dir from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_new_schema_08-31-23-1 to ../MIMIC_ESD_new_schema_08-31-23-1
Loading events from ../MIMIC_ESD_new_schema_08-31-23-1/events_df.parquet...
Loading dynamic_measurements from ../MIMIC_ESD_new_schema_08-31-23-1/dynamic_measurements_df.parquet...


In [3]:
def has_event_type(type_str: str) -> pl.Expr:
    has_event_type = pl.col("event_type").cast(pl.Utf8).str.contains(type_str)
    # has_event_type = event_types.str.contains(type_str)
    return has_event_type

In [6]:
def validate_query(config_path, data_path, ESD_data, samples=10, verbose=False, seed=42):
    cfg = main.load_config(config_path)
    df_data = main.generate_predicate_columns(cfg, ESD_data, verbose=verbose)

    df_result = main.query_task(config_path, ESD_data, verbose=verbose)

    if df_result.shape[0] == 0:
        print("No results found.")
        return

    try:
        validation = df_result.sample(samples, seed=seed)
        validation = validation.to_pandas()
    except:
        validation = df_result.to_pandas()

    # validation = df_result.filter(df_result['subject_id'] == 1916).to_pandas()

    for i, row in validation.iterrows():
        if i % 100 == 0:
            print(f"Validating row: {i}/{len(validation)}...")
        #filter ESD_data for the subject_id
        subject_id = row['subject_id']
        filtered_data = df_data.filter(pl.col("subject_id") == subject_id)

        if verbose:
            print('Checking subject_id:', subject_id)

        #sort the values of every column that has 'timestamp' in its name
        timestamps = []
        for col in validation.columns:
            if 'timestamp' in col:
                timestamps.append((row[col], col))
        timestamps.sort(key=lambda x: x[0])

        filtered_data = filtered_data.filter(pl.col("timestamp") >= timestamps[0][0])
        filtered_data = filtered_data.filter(pl.col("timestamp") <= timestamps[-1][0])

        trigger_event = f'is_{cfg.windows.trigger.start}'
        assert filtered_data.filter(pl.col("timestamp") == row['trigger/timestamp']).select(trigger_event).to_pandas().values.flatten()[0] == 1
        
        after_trigger = False
        for i in range(len(timestamps)-1):
            window = timestamps[i]
            name = window[1].split('/')[0]
            if verbose:
                print(f"Checking window: {name}->{timestamps[i+1][1].split('/')[0]}")
            if name == 'trigger':
                after_trigger = True
            if after_trigger:
                name = timestamps[i+1][1].split('/')[0]

            if verbose:
                print(timestamps[i][0], timestamps[i+1][0])
                print(row[f'{name}/window_summary'])
            window_data = filtered_data.filter(pl.col("timestamp") > timestamps[i][0])
            window_data = window_data.filter(pl.col("timestamp") <= timestamps[i+1][0])

            sum_counts = window_data.sum()
            if verbose:
                display(sum_counts)

            for predicate in row[f'{name}/window_summary']:
                count = row[f'{name}/window_summary'][predicate]
                if not count:
                    count = 0
                assert sum_counts.select(predicate).to_pandas().values.flatten()[0] == count, (subject_id, timestamps[i][0], timestamps[i+1][0], row[f'{name}/window_summary'], sum_counts)

In [8]:
# config_path = '../sample_configs/inhospital_mortality.yaml'
# config_path = '../sample_configs/abnormal_lab.yaml'
# config_path = '../sample_configs/imminent_mortality.yaml'
# config_path = '../sample_configs/intervention_weaning.yaml'
# config_path = '../sample_configs/long_term_incidence.yaml'
config_path = '../sample_configs/readmission_risk.yaml'
validate_query(config_path, data_path, ESD_data, samples=10, verbose=True, seed=42)

Added predicate column is_admission.
Added predicate column is_discharge.
Loading config...

Generating predicate columns...

Added predicate column is_admission.
Added predicate column is_discharge.

Building tree...
trigger
┗━━ input
    ┗━━ target


12127 subjects (14623763 rows) were excluded due to trigger event: admission.


Querying...


Querying subtree rooted at input...
88 subjects (119 rows) were excluded due to constraint: [(col("is_discharge")) >= (1)].
12 subjects (12 rows) were excluded due to constraint: [(col("is_discharge")) <= (1)].


Querying subtree rooted at target...


Done.

Validating row: 0/10...
Checking subject_id: 1989
Checking window: trigger->input
2114-05-19 12:32:00 2114-05-24 10:10:00
{'is_admission': 0, 'is_discharge': 1, 'is_any': 64}


subject_id,timestamp,is_admission,is_discharge,is_any
i64,datetime[μs],i32,i32,i32
127296,null,0,1,64


Checking window: input->target
2114-05-24 10:10:00 2114-06-23 10:10:00
{'is_admission': None, 'is_discharge': None, 'is_any': None}


subject_id,timestamp,is_admission,is_discharge,is_any
i64,datetime[μs],i32,i32,i32
0,null,0,0,0


Checking subject_id: 3738
Checking window: trigger->input
2174-09-06 07:41:00 2174-09-08 16:00:00
{'is_admission': 0, 'is_discharge': 1, 'is_any': 1}


...

subject_id,timestamp,is_admission,is_discharge,is_any
i64,datetime[μs],i32,i32,i32
320,null,1,1,2


Checking subject_id: 2836
Checking window: trigger->input
2172-12-02 00:26:00 2172-12-03 14:01:00
{'is_admission': 0, 'is_discharge': 1, 'is_any': 1}


subject_id,timestamp,is_admission,is_discharge,is_any
i64,datetime[μs],i32,i32,i32
2836,null,0,1,1


Checking window: input->target
2172-12-03 14:01:00 2173-01-02 14:01:00
{'is_admission': 1.0, 'is_discharge': 1.0, 'is_any': 2.0}


subject_id,timestamp,is_admission,is_discharge,is_any
i64,datetime[μs],i32,i32,i32
5672,null,1,1,2
